# <font color = 'red'> Analisando Sentimentos das reclamações</font>

In [1]:
!pip install -q textblob

In [27]:
!pip install transformers

     --------------------------------------- 10.0/10.0 MB 13.1 MB/s eta 0:00:00
     -------------------------------------- 447.5/447.5 kB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 13.8 MB/s eta 0:00:00
     -------------------------------------- 286.1/286.1 kB 8.6 MB/s eta 0:00:00
     -------------------------------------- 179.6/179.6 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.7.1
    Uninstalling fsspec-2022.7.1:
      Successfully uninstalled fsspec-2022.7.1


In [28]:
# Imports
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import AutoTokenizer, AutoModel
import torch

from textblob import TextBlob

import warnings
warnings.filterwarnings('ignore')

In [39]:
dados_sentimento = pd.read_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/dados_financeiros_sentimentos.csv',
                       sep =',')

In [40]:
dados_sentimento.sample(5)

,Empresa,Relato,Avaliacao
1354,Banco Inter (Banco Intermedium),tenho uma divida com um cartao de credito e su...,4
1403,Banco do Brasil,pretendo renegociar a pendência. porém com os ...,1
205,Caixa Econômica Federal,29102024 1641 55 21 xxxxxx981 caro a aluno a ...,1
1344,Caixa Econômica Federal,informo que a caixa econômica não reduz os jur...,2
1609,Banco do Brasil,já fui na agência do banco do brasil duas veze...,1


In [41]:
dados_sentimento.shape

(1920, 3)

In [35]:
### Função para obter subjetividade
def subjetividade(text):
 blob = TextBlob(text)
 return blob.sentiment.subjectivity

In [36]:
# Função para obter polaridade
def polaridade(text):
 blob = TextBlob(text)
 return blob.sentiment.polarity

In [37]:
# Adicionando no DF
dados_sentimento['Subjetividade'] = dados_sentimento['Relato'].apply(subjetividade)
dados_sentimento['Polaridade'] = dados_sentimento['Relato'].apply(polaridade)

In [42]:
dados_sentimento.head(10)

,Empresa,Relato,Avaliacao
0,Caixa Econômica Federal,não consigo crédito pra fazer um empréstimo me...,3
1,Nubank,ja houve o pagamento da negociacao,2
2,Nubank,exclusao do nome retirar negativacao do nome s...,1
3,C6 Bank,sou cliente do c6 bank há tempos sempre manten...,1
4,Nubank,gostaria de reclamar sobre essa dívida quer es...,5
5,Banco Itaú Unibanco,o itaú financiamentos fez constar na scr uma l...,1
6,Nubank,eu fiz um acordo para pagamento da minha dívid...,3
7,Banco Itaú Unibanco,primeiramente quero agradecer o banco itau poi...,5
8,Nubank,reclamação nubank gostaria de registrar minha ...,1
9,Caixa Econômica Federal,desejo participar do mutirão de renegociação.,1


In [44]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Carregar o modelo pré-treinado para análise de sentimento
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Função para prever a polaridade e intensidade com RoBERTa-BR
def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    scores = outputs.logits.softmax(dim=1).detach().numpy()
    
    # Exemplos de interpretação (ajustar conforme os rótulos do modelo que está usando)
    sentiment = "positivo" if scores[0][1] > scores[0][0] else "negativo"
    intensity = scores[0][1]  # Considerando que pontuações mais altas indicam mais intensidade
    
    return sentiment, intensity

# Aplicar o modelo para obter polaridade e intensidade
dados_sentimento['Polaridade'], dados_sentimento['Intensidade'] = zip(*dados_sentimento['Relato'].apply(get_sentiment))

# Exibir o resultado
print(dados_sentimento)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                      Empresa  \
0     Caixa Econômica Federal   
1                      Nubank   
2                      Nubank   
3                     C6 Bank   
4                      Nubank   
...                       ...   
1915      Banco Itaú Unibanco   
1916      Banco Itaú Unibanco   
1917                  C6 Bank   
1918           Banco Bradesco   
1919  Caixa Econômica Federal   

                                                 Relato  Avaliacao Polaridade  \
0     não consigo crédito pra fazer um empréstimo me...          3   negativo   
1                    ja houve o pagamento da negociacao          2   negativo   
2     exclusao do nome retirar negativacao do nome s...          1   negativo   
3     sou cliente do c6 bank há tempos sempre manten...          1   negativo   
4     gostaria de reclamar sobre essa dívida quer es...          5   negativo   
...                                                 ...        ...        ...   
1915  solicitando a retirada da rest

In [46]:
dados_sentimento.sample(20)

,Empresa,Relato,Avaliacao,Polaridade,Intensidade
1307,Banco Bradesco,prezadoa solicito que entrem em contato o quan...,5,negativo,0.409989
1311,Banco Itaú Unibanco,preciso que excluem está dívida caduca em meu ...,2,negativo,0.463328
48,Nubank,solicitei ao banco o descritivo de evolução da...,1,negativo,0.463280
1895,Caixa Econômica Federal,acredito que os juros mensais estão muito elev...,1,negativo,0.409643
1292,Caixa Econômica Federal,fiz no dia 17102024 uma locação de um carro na...,5,negativo,0.412969
1183,Banco Santander,eu estava com dividas nos bancos santander i...,5,negativo,0.404824
990,Banco Inter (Banco Intermedium),tive meu celular furtado no dia 21024 na mesma...,5,negativo,0.409019
844,Nubank,venho por meio desta solicitar uma renegociaçã...,1,negativo,0.412940
1201,Banco do Brasil,recebi um consócio cedente de adesão número 4....,5,negativo,0.428601
494,Caixa Econômica Federal,boa tarde vi que tenho pendências na caixa eco...,1,negativo,0.426547


In [47]:
dados_sentimento.Polaridade.value_counts()

Polaridade
negativo    1892
positivo      28
Name: count, dtype: int64

In [48]:
dados_sentimento[dados_sentimento['Polaridade'] == 'positivo']

,Empresa,Relato,Avaliacao,Polaridade,Intensidade
16,Nubank,em 04112024 verifiquei a o extrato cartão de ...,1,positivo,0.510398
59,Nubank,em 03112024 verifiquei ao registrato na qual c...,3,positivo,0.513293
192,Nubank,em 31102024 verifiquei a o extrato da conta co...,1,positivo,0.503650
248,Nubank,verifiquei junto ao serasa uma dívida em abert...,1,positivo,0.519736
263,Banco do Brasil,em 31102024 verifiquei a o extrato cartão de c...,1,positivo,0.504479
403,Banco do Brasil,por várias vezes fui a agência em busca de inf...,1,positivo,0.500896
438,Banco do Brasil,que de baixa a registros anteriores a cinco an...,1,positivo,0.503873
481,Nubank,em 14 de abril de 2024 verifiquei a o extrato ...,1,positivo,0.505893
543,Nubank,em 29102024 verifiquei a o extrato extrato ca...,1,positivo,0.505728
555,Nubank,em 2910 verifiquei a o extrato do cartão de cr...,1,positivo,0.503945
